# Challenge 5:  127 큐비트를 사용해 봅시다


이전 도전 과제들을 모두 해결한 것을 축하합니다! 여러분은 동적회로를 사용할 수 있음을 보여주었습니다. 
지금까지 이룬 것들에 대한 보상으로, 여러분은 이제 IBM Quantum의 127 큐비트 장비를 사용할 수 있습니다.

다시 한 번 말씀드리지만, 챌린지 완료 배지를 획득하려면 5개 랩 중 4개를 완료해야 합니다. 그 이유는 이번 실습이 (a) 기술적으로 도전적이며 (b) 최첨단 하드웨어를 사용해야 하기 때문입니다. 참가자 모두가 사용할 수 있도록 100큐비트 이상의 장치를 제공하기 때문에 회로를 실행하기까지 긴 대기시간이 필요 할 수도 있습니다. 여러분 모두 사용해 볼 수 있도록 운영진이 무대뒤에서 최선을 다해 가능한 모든 것을 하고 있지만, 올해의 챌린지 참가자 숫자가 기록적이기 때문에 불편함을 겪을 수도 있습니다. 따라서 모두에게 친절히 대해 주시고, 작업 대기열이 막하지 않도록 같은 작업을 반복해서 제출하지 말아 주시길 부탁합니다. 

이제 심호흡을 하고 시작해 봅시다.


여러분은 이미 앞에서 실제 물리적 장비를 사용하는 것 자체가 도전 과제를 불러온 다는 것을 확인했습니다.
이런 점은 여러분이 더 많은 숫자의 큐비트를 사용하는 경우 두드러 지게 되는데, 이것은 회로를 실행하기 위한 경로가 길어지며 이것이 너무 많은 오류를 만들어 내지 않도록 해야 하기 때문입니다. 

이번 도전 과제에서, 여러분은 GHZ 상태라 불리우는 완벽하게 얽힌 127 큐비트의 상태를 영리하게 만들어 내야 합니다. 

그런 다음 좋은 54큐비트 GHZ 상태를 만들어 내기 위해 이 127큐비트 GHZ 상태를 사용해야 하는지에 있어서 오류 수정에 대한 이전의 지식을 어떻게 적용할 수 있는지에 대한 가이드를 제공하겠습니다.

*노트: 여기서는 같은 레지스터를 홀수 및 짝수 큐비트에 대해 사용합니다. 특히 회로의 깊이를 수동으로 최적화하려는 경우 초기 127 큐비트 GHZ 상태를 생성하는 것이 훨씬 더 복잡해지기 때문에 이 방식을 선택하였습니다.*

In [ ]:
# Importing all the parts we need
from typing import List, Optional

from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.result import marginal_counts

import warnings

warnings.filterwarnings("ignore")

import math

pi = math.pi

# Preparing registers
quantum_register = QuantumRegister(127)
classical_register = ClassicalRegister(127)

# For simplicity we map the physical qubits to the logical qubits directly using the same number.
initial_layout = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    49,
    50,
    51,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    100,
    101,
    102,
    103,
    104,
    105,
    106,
    107,
    108,
    109,
    110,
    111,
    112,
    113,
    114,
    115,
    116,
    117,
    118,
    119,
    120,
    121,
    122,
    123,
    124,
    125,
    126,
]
# The "even"" qubits will be used for the 54 qubit GHZ-state
ghz_qubits = [
    0,
    2,
    4,
    6,
    8,
    10,
    12,
    18,
    20,
    22,
    24,
    26,
    28,
    30,
    32,
    37,
    39,
    41,
    43,
    45,
    47,
    49,
    51,
    56,
    58,
    60,
    62,
    64,
    66,
    68,
    70,
    75,
    77,
    79,
    81,
    83,
    85,
    87,
    89,
    94,
    96,
    98,
    100,
    102,
    104,
    106,
    108,
    114,
    116,
    118,
    120,
    122,
    124,
    126,
]
# The "odd" qubits will be used as the stabilizers
stabilizer_qubits = [
    1,
    3,
    5,
    7,
    9,
    11,
    14,
    15,
    16,
    17,
    19,
    21,
    23,
    25,
    27,
    29,
    31,
    33,
    34,
    35,
    36,
    38,
    40,
    42,
    44,
    46,
    48,
    50,
    52,
    53,
    54,
    55,
    57,
    59,
    61,
    63,
    65,
    67,
    69,
    71,
    72,
    73,
    74,
    76,
    78,
    80,
    82,
    84,
    86,
    88,
    90,
    91,
    92,
    93,
    95,
    97,
    99,
    101,
    103,
    105,
    107,
    109,
    110,
    111,
    112,
    115,
    117,
    119,
    121,
    123,
    125,
]

## 1 단계: 커다란  GHZ-상태 생성

아주 많은 양자 알고리듬들을 여러 큐비트들을 서로 얽는 것을 필요로 합니다.가장 흔하게 사용되는 것 중 하나는 일반화된 GHZ-상태로, $\alpha = \beta$인 $(\alpha |00...0\rangle + \beta |11...1\rangle)$ 상태입니다.

GHZ 상태는 완전히 얽혀 있고 임의의 수의 큐비트로 일반화될 수 있으므로 127 큐비트 GHZ 상태를 생성해 봅시다! 
측정을 하게 되면 GHZ 상태에 사용된 큐비트 수에 관계없이 모든 큐비트가 0이거나 모든 큐비트가 1입니다.

이곳의 도전 과제는 이론적으로 127 큐비트를 얽는 것이 아니라 실제 장비에서 구현하는 것입니다. 즉, 하드웨어의 레이아웃을 고려해야 한다는 의미입니다 (따라서 직접 연결되어 있지 않은 큐비트들 사이에 CX를 사용하면 안됩니다). 그리고 추가적으로 노이즈를 줄이기 위해 회로의 깊이를 낮추기 위해 노력해야 합니다.


### 연습 문제 1
127 큐비트 GHZ-상태를 127-큐비트 장치인 ibm_sherbrook에 하다마드 게이트와 CNOT 게이트 만을 사용하여 구현하시오. 가장 낮은 회로의 깊이로 이 상태를 구현해 봅시다.

*힌트: 이상적으로 모든 큐비트가 연결되어 있는 장비에서 최소의 회로 깊이로 구현하는 방법을 먼저 생각해 보는 것이 도움이 될 것입니다.*

In [ ]:
def generate_ghz127():
    qc = QuantumCircuit(quantum_register, classical_register)

    ####### your code goes here #######

    return qc


ghz_circuit = generate_ghz127()
print(ghz_circuit.depth())

In [ ]:
# Submit your circuit

from qc_grader.challenges.spring_2023 import grade_ex5a

grade_ex5a(ghz_circuit)

## 2 단계: 사이즈를 절반으로 줄입니다

이제 위에서 만든 GHZ-상태 중 짝수의 큐비트만을 사용하여 크기를 줄인 GHZ-상태를 만들어 보겠습니다. 이것을 위해, 홀수의 큐비트들의 얽힘을 풀어서 홀수의 큐비트들을 측정하더라도 GHZ-상태에 영향을 미치지 않도록 합니다.

홀수 큐비트의 얽힘을 푼 후에는, 오류 정정 부분에서 해 본 방식으로 안정자(stabilizer)를 구현하기 위해 사용할 것입니다. 따라서 연결된 두 큐비트의 값이 같으면 0이어야 하고 값이 다르면 1이어야 합니다.

그 다음 이 안정자들을 줄어든 GHZ-상태의 오류 정정을 위해 사용할 수 있으며, 이에 대해서는 나중에 다시 설명하도록 합니다.

### 연습 문제 2
만들어진 GHZ-회로의 홀수 큐비트들의 얽힘을 풀어 안정자를 만듭시다. 마찬가지로 가장 작은 회로의 깊이에 도전해 보세요.

*힌트: 위에서 GHZ-상태를 만든 방법을 고려해야 합니다. 얽힘을 푸는 방법은 얽힘이 어떻게 만들어졌는지에 따라 다릅니다.*

In [ ]:
def deentangle_qubits():
    qc = QuantumCircuit(quantum_register, classical_register)

    ####### your code goes here #######

    return qc


unentangle_circuit = deentangle_qubits()
print(unentangle_circuit.depth())

complete_circuit = ghz_circuit.compose(unentangle_circuit)

In [ ]:
# Submit your circuit

from qc_grader.challenges.spring_2023 import grade_ex5b

grade_ex5b(complete_circuit)

이제 모든 홀수 큐비트가 짝수 큐비트와 얽히지 않았기 때문에 GHZ-상태를 건드리지 않고 안정자를 측정할 수 있습니다.

In [ ]:
# Measuring stabilizers this can also be used in post processing to see what went wrong.


def measure_stabilizers():
    qc = QuantumCircuit(quantum_register, classical_register)
    qc.measure(stabilizer_qubits, stabilizer_qubits)
    return qc


stabilizer_circuit = measure_stabilizers()

이 측정 결과들은 54 큐비트의 GHZ-상태를 개선하는데 사용될 수 있지만 지금은 생략하도록 하겠습니다.

홀수 큐비트들은 측정한 후 재설정할 수 있으며, 홀수 큐비트를 잠재적인 보조큐비트로 사용하여 알고리듬에서 54 GHZ-상태를 사용할 수 있습니다.

하지만, GHZ-상태를 사용하기 전, 먼저 만들어진 GHZ-상태가 얼마나 훌륭한지 테스트 해봅시다.

그러기위해, 먼저 54 큐비트 GHZ-상태를 측정해 봅시다. 

In [ ]:
# Measuring the GHZ qubits


def measure_ghz():
    qc = QuantumCircuit(quantum_register, classical_register)
    qc.measure(ghz_qubits, ghz_qubits)
    return qc


measure_circuit = measure_ghz()

이제 모든 것을 한나로 모아, 테스트 해봅시다.

In [ ]:
# Everything together

simple_ghz = (
    ghz_circuit.compose(unentangle_circuit)
    .compose(stabilizer_circuit)
    .compose(measure_circuit)
)

## 3 단계: 실제 하드웨어에 실행 준비

이제 만들어진 GHZ-상태 회로를 실제 장비에 실행할 준비가 되었고, 회로를 실행할 실제 127 큐비트 장비가 마련되어 있습니다. 

이 세상에서 127 큐비트 장비를 사용할 수 있는 사람은 그리 많지 않고, 여러분은 곧 그들 중 하나가 될 것입니다! 양자 컴퓨팅에 대한 여러분의 작은 발걸음이자 여러분의 CV의 거대한 도약이 될 것입니다.  

농담인것 아시죠? :-P

-----

먼저, 필요한 모든 것을 준비해 봅시다.

In [ ]:
# Importing provider and getting access to the system
from qiskit_ibm_provider import IBMProvider
from qiskit import transpile


provider = IBMProvider()


hub = "YOUR_HUB"
group = "YOUR_GROUP"
project = "YOUR_PROJECT"

backend_name = "ibm_sherbrooke"
backend = provider.get_backend(backend_name, instance=f"{hub}/{group}/{project}")

number_of_shots: int = 1024

먼저 실제 장치에 회로가 실행될 수 있도록 양자 회로를 트랜스파일합니다. 처음에 회로를 만들 때 이미 물리적 장치를 염두에 두고 이미 연결된 큐비트들만 사용했기 때문에 너무 많은 변경은 발생하지 않을 것입니다. 

만약 그렇게 하지 않았다면 다시 돌아가서 고치기에 좋은 타이밍입니다, 혹은 디스코드로 가서 다른 사람들과 어떻게 구현할 수 있을 지 상의해 봅시다.

In [ ]:
# First we transpile the GHZ-state for the actual device
qc_transpiled = transpile(simple_ghz, backend, initial_layout=initial_layout)

이제 만들어진 회로는 127 큐비트 장치에서 실행 될 수 있게 되었습니다! 
다음에 찾기 쉽도록 태그를 붙여 둡시다. 

장비에 잡을 실행하는 것은 얼마나 많은 사람들이 잡을 실행하기 위해 대기열에 대기중인지에 따라 결정됩니다. 혹시 오늘 커피는 드셨나요? 혹시 커피 대신 차를 마시는 타입이신가요? 넷플릭스에 있는 영화를 보면서 기다리는건 어때요? 

In [ ]:
# Now we can run the job
# We add memory=true to be easier able to analyse  how good the result were and the tags to make it easier to find it later.
job = backend.run(
    qc_transpiled,
    shots=number_of_shots,
    memory=True,
    job_tags=["ghz_state", "spring_challenge"],
)

job_id = job.job_id()
print(job_id)

작업을 백엔드에 전송해 둔 후 나중에 돌아와서 작업의 결과를 가져오려는 경우, IBM Quantum Computing 홈페이지(위에 사용된 태그 포함)에서 해당 작업을 찾고 작업의 이름(ID)을 복사한 다음 아래의 *job_id*를 해당 작업으로 대체할 수 있습니다. 작업 ID는 다음과 같이 보여야 합니다. *ch36cf1plju56fajrqg*

혹은 위의 코드셀에서 저장한 job_id를 바로 사용할 수도 있습니다.

In [ ]:
job = provider.retrieve_job(job_id)
job.status()

In [ ]:
# Getting the data of the job for testing
data = retrieve_job.result().get_memory()

## 4 단계: GHZ-상태 테스트

실제 양자 백엔드에서 여러분의 GHZ-상태 회로를 실행한 결과를 가져온 후, 결과가 얼마나 좋은지 확인하고 싶을 것입니다. 
아직 오류 완화나 오류 보정을 적용하지 않았으므로, 측정된 결과는 더 개선될 수 있습니다.

측정된 결과의 품질을 테스트 할수있는 방법은 여러가지가 있으며, 물론 결과를 바로 들여다 볼 수도 있지만, GHZ-상태 회로의 큐비트 들이 바로 옆으로 이어져 있지 않기때문에 살펴보기에 다소 번거로울 수 있습니다. 

따라서 회로 실행 결과의 품질이나, GHZ-회로의 오류에 대해 알려주는 함수를 만드는 것이 도움이 될 것입니다.

### 연습 문제 3
GHZ-상태의 품질을 검증할 함수를 만들자

이 곳에서 여러분이 회로를 테스트하는 방식에서 여러분에게 와닿는 방식으로 구현해보는 자유를 갖길 원합니다. 여기에서 중요한 것은 **나은 품질**의 GHZ-상태란 여러분이 만들 함수의 결과값이 낮아짐을 의미해야 합니다. 

*힌트: 이 함수는, 여러분이 구현하여 얻은 결과를 검증하기 위해 만들어 져야 합니다. 따라서, GHZ-상태에 사용된 큐비트들 만을 검증의 대상으로 삼아야 합니다*

In [ ]:
# A function to test the quality of a GHZ-state. The lower the better
def test_ghz(data):
    ghz_qubits = [
        0,
        2,
        4,
        6,
        8,
        10,
        12,
        18,
        20,
        22,
        24,
        26,
        28,
        30,
        32,
        37,
        39,
        41,
        43,
        45,
        47,
        49,
        51,
        56,
        58,
        60,
        62,
        64,
        66,
        68,
        70,
        75,
        77,
        79,
        81,
        83,
        85,
        87,
        89,
        94,
        96,
        98,
        100,
        102,
        104,
        106,
        108,
        114,
        116,
        118,
        120,
        122,
        124,
        126,
    ]

    ####### your code goes here #######


test_ghz(data)

이제 결과의 등급을 매기는 숫자를 갖게 되었습니다. 

계속하기 전에 여러분의 방법에 대해 생각해 봅시다. 
- 정말 좋은 상태에 대한 값은 무엇일까요? 
- 정말 나쁜 상태에 대해서는 어떤 값이 나올까요?
- 그리고 그런 나쁜 상태는 어떻게 보일까요?
- 다른사람이 이것을 쉽게 이해할 수 있는 함수를 어떻게 설계할 수 있을 까요? 

이제, 여러분의 검증 함수를 검증해 보고, 함수가 제 기능을 잘 하고 있는지 확인해 봅시다 ( 낮은 오류를 지닌 좋은 상태에는 낮은 점수를, 오류가 많은 상태에는 높은 점수를 주어야 합니다)


In [ ]:
# Submit your circuit

from qc_grader.challenges.spring_2023 import grade_ex5c

# Since we test here a function, we do not need brackets after test_ghz, since the input is the function
grade_ex5c(test_ghz)

이제, GHZ-상태의 결과를 얻었다면, 남은 질문은 어떻게 개선할 수 있는가? 입니다.

첫번째 단계는 회로의 깊이를 어떻게 더 줄이기 위해 노력해 보는 것입니다. 여러분의 트랜스파일된 회로의 깊이를 확인해 봅시다:


In [ ]:
qc_transpiled.depth()


트랜스파일 과정에서 회로의 깊이가 두 배이상 늘어난 것을 알 수 있습니다. 

이 회로가 어떻게 생겼는지, 어디에서 추가적인 깊이가 발생한 것인지 확인해 봅시다.

In [ ]:
qc_transpiled.draw()

여기에서 확인할 수 있는 것은 하다마드 게이트와 CX 게이트가 트랜스파일 되어 다른 게이트로 구현된 것입니다. 하다마드 게이트는 어떤 IBM Quantum의 장비도 바로 이 게이트를 실행하지 못하고 3개의 회전 게이트로 구현하기 때문에 놀라운 결과가 아닙니다. 

반면에, 볼수 있는 ECR-게이트는 새로운 것이며 일부의 장비에서만 사용되는 것입니다. 이것은 얽힘 게이트의 부류로 CX-게이트와 유사하지만 실제 동작은 약간 다릅니다. 하지만 CX-게이트는 하나의 ECR-게이트와 국소 회전을 사용해 구현됩니다. 


ECR-게이트에 대한 자세한 설명은 이곳에서 확인할 수 있습니다:
https://qiskit.org/documentation/stable/0.39/stubs/qiskit.circuit.library.ECRGate.html

이제, ECR-게이트가 대신 사용된 것을 감안하여, 여러분은 회로의 깊이를 개선할 수 있을까요? 

이것은 연습 문제는 아닙니다, 하지만 여러분이 도전해 볼 수 있는 주제입니다. 만약 도전하겠다면, 회로의 깊이를 줄이는 것은 오류율을 낮추는 가장 좋은 방법이기도 합니다.

GHZ-상태를 더 좋게 만드는 데 사용할 수 있는 다른 방법을 알고 싶다면 다음의 내용을 읽어 보길 바랍니다. 이어지는 도전은 보너스일 뿐이며 필수 도전과제는 아닙니다. 오류 정정이 어떻게 사용될 수 있는지를 보여주기 위한 방법에 대한 전략을 이야기 할 수 있지만, 오류 정정이 어려운 이유도 또한 알게 될 것입니다.

여러분이 실습을 계속할지 말지에 상관없이, 여러분 스스로에게 큰 박수를 쳐 주도록 합시다: 여러분은 방금 박 IBM Quantum Spring Challenge 2023을 마쳤습니다! 디스코드에 가서 모두에게 자랑해 봅시다! 거기에서 만나요! 

# 보너스: 오류를 정정하는 방법


오류 정정은 여전히 활발하고 중요한 연구 주제입니다. 따라서 실제 장비의 오류를 정정하는 것은, 아주 간단한 GHZ-상태와 같은 예제를 사용했다 하더라도, 그리 직관적이지 않고 여러분에게 몇가지 조언과 힌트, 사용할 수 있는 잠재적 방법들을 소개함으로서 여러분만의 방법을 만들어 낼 수 있는 기회를 제공하고자 합니다. 

## 1 단계:  실제 상태는 얼마나 좋은 것일까?

오류를 정정하기 전, 먼저 상태가 좋다는 것에 대해 생각해 보아야 합니다, 그리고 이것조차 생각보다 쉽지 않습니다.


여러분의 GHZ-상태가 얼마나 좋은지 테스트하는 가장 간단한 방법은, 짝수의 큐비트 들이 얼마나 많이 주요 결과와 다른 결과를 갖는지 확인하는 것입니다. 아마 이것이 여러분이 위에서 주로 사용한 방법일 것입니다. 


하지만, 이것은 꼭 발생한 오류의 숫자와 관련이 있다고 말할 수 없습니다. 그리고 누군가는 GHZ-상태의 품질에는 발생한 오류의 숫자가 더욱 중요하다고 말 할지도 모릅니다.


**예제 A** 이 생각을 묘사해 봅시다: 10개의 큐비트가 일렬로 연결되어 있고 이것이 GHZ-상태를 구성하고 있다고 생각해 봅시다: 

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9

만약 오류가 readout에서 발생하는 것이 아니라, 큐비트 4와 5를 얽는 게이트에서 발생했다고 한다면, 출력 결과는 다음과 같을 것입니다:

0000011111 

이것은 "얼마나 많은 큐비트가 같은 값을 가지고 있는지"를 측정으로 얻고자 할때의 최악의 가능한 경우일 것입니다. 

이런 이유로, 더 많은 오류가 발생한 결과는, 실제로 "동일한 결과를 갖는 큐비트의 수"에 대해서는 더 나은 측정으로 여겨질 수 있지만, 덜 유용할 수 있습니다.

만약 위의 상태가 만들어 졌고, 여러분이 큐비트들 사이의 안정자를 측정 결과를 가지고 있다면, 큐비트4와 5사이에 1의 값을 갖게 될 것입니다. 그리고 그 오류를 정정하기 위해서는 아마도 큐비트 5부터 9까지의 비트를 반전해야 할 것입니다. 


여러분의 경우에 오류가 어떻게 보일지에 대한 첫번째 아이디어를 얻기 위해, 실제 장치에서 실행되어 받은 결과의 원시 데이터를 살펴보는 것이 도움이 될 것이며, 여러분이 GHZ-상태를 구현한 방식도 살펴 보는 것이 도움이 될 것입니다. 


 **예제 A**에서는 다음과 같은 오류 정정을 구현하게 될 것입니다:

In [ ]:
# Simple idea for Example A find where the error happened and flip accordingly
def correct_ghz():
    qc = QuantumCircuit(quantum_register, classical_register)

    with qc.if_test((classical_register[9], 1)):
        qc.x(quantum_register[9])

    with qc.if_test((classical_register[8], 1)):
        qc.x(quantum_register[8])

    with qc.if_test((classical_register[8], 1)):
        qc.x(quantum_register[9])

    with qc.if_test((classical_register[7], 1)):
        qc.x(quantum_register[9])

    with qc.if_test((classical_register[7], 1)):
        qc.x(quantum_register[8])

    with qc.if_test((classical_register[7], 1)):
        qc.x(quantum_register[7])

    # ...

    # qc.barrier()
    return qc


correcting_circuit = correct_ghz()

그리고 이 방식을 여러분의 54 GHZ-상태 회로에 적용한다면, 모든 것을 하나로 모아 다음과 같이 작성합니다:


In [ ]:
# Everything together corrected

error_corrected_ghz = (
    ghz_circuit.compose(unentangle_circuit)
    .compose(stabilizer_circuit)
    .compose(correcting_circuit)
    .compose(measure_circuit)
)

##  2 단계: 왜 단일 안정자를 고려하는 것 만으로는 충분하지 않을까

오류 정정 연습에서, 단일 비트를 사용하거나 안정자 큐비트 측정의 전체 비트스트링을 사용하여 작업을 조건적 연산을 사용할 수 있음을 확인했습니다.


여러분의 안정자 측정결과가 54비트 길이를 지니기 때문에, 모든 비트 문자열에 대해 조건적 연산을 적용하는 것은 2^54의 각기 다른 문자열을 고려해야 하기 때문에 실행 가능해 보이지 않습니다. 


반면에, 단일 비트만을 고려한다면 그 또한 불가능하거나 지나치게 복잡할 수 있습니다.


**예제 B**: 10개의 큐비트가 다음과 같이 일직선으로 연결되어 있고 GHZ-상태를 구성하고 있습니다: 
0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9

이제, 측정 결과가 0000010000라고 가정해 봅시다. 

만약 이 경우, 안정자의 측정결과가 있다면, 1의 왼쪽과 오른쪽에 있는 비트의 결과는 모두 1이 됩니다.

이 경우 단일 안정자를 보는 것만으로는 큐비트 번호 5만 뒤집으면 된다는 것을 알기에 충분하지 않습니다.

따라서 이 예제의 경우 안정자 측정 결과들에 함수를 적용하고 동적 회로의 조건으로 이 결과들을 사용하는 것이 이상적일 것입니다. 

구현할 수는 있지만, Qiskit이 아닌 Open QASM3 코드로 작성되어야 하며, 이번 연습에서 다루지는 않겠습니다. 
만약 동적회로를 OpenQASM3으로 사용하는 것게 관심이 있다면, 더 많은 정보를 이곳에서 찾아볼 수 있습니다: 

https://quantum-computing.ibm.com/services/resources/docs/resources/manage/systems/dynamic-circuits/Dynamic-Circuits-basics-with-OpenQASM3


아래에 Open QASM3 코드가 어떻게 생겼는지를 보여주는 간단한 예제가 있습니다: 

In [ ]:
from qiskit import qasm3, QuantumCircuit, transpile

# Creating a bell circuit
qc_bell = QuantumCircuit(2, 2)
qc_bell.h(0)
qc_bell.cx(0, 1)
qc_bell.measure(0, 0)
qc_bell.measure(0, 1)

# Transpiling it for our device (as above it does not have the H- and CX- Gates)
qc_bell = transpile(qc_bell, backend)

# Generate qasm3 code before we can print it
exporter = qasm3.Exporter(
    includes=[], disable_constants=True, basis_gates=backend.configuration().basis_gates
)
print(qasm3_bell := exporter.dumps(qc_bell))

# Draw a circuit as comparison
qc_bell.draw(output="mpl", idle_wires=False)

## 3 단계: 어떻게 더 많은 오류를 일으키지 않을 수 있을까 


오류가 발생하는 원인은 다양합니다. 시간이 지나며 발생하는 결잃음(Decoherence)와 CX-게이트를 비롯한 얽힘 게이트에서 발생하는 오류도 그 중 두가지에 해당합니다. 두가지 모두 안정자에서도 발생할 수 있으며, 큰 목적을 달성하기 위해 이것을 정정하려는 시도로 인해 더 많은 오류가 발생하지 않도록 주의해야 합니다. 일반적인 오류율이  낮아질 미래에는 덜 중요한 문제가 되겠지만 여전히 중요하게 생각해야 할 문제입니다. 

이것은 무엇을 의미하는 것일까요?

안정자를 만들고 측정할 때를 생각해 볼 수 있습니다. 더 이상 다음 큐비트를 얽히는데 사용하지 않게 되는 직후에 얽힘을 풀 것인가? 전체 회로가 얽힐 때까지 기다려야 할까요? (물론 이것은 우리가 정정하고자 하는 오류의 종류에 따라 다릅니다). 또한 안정자를 재설정하고 나중에 동일한 큐비트를 사용하여 새 안정자를 만들고 오류 정정을 시도하는 두개의 단계를 사용할 수도 있습니다. 


그리고 (안정자를 통해) 큐비트를 얽을때 발생하는 에러의 확률을 생각해 볼 수 있으며, 이것을 안정자의 얽힘을 풀때 발생하는 에러와 비교해 볼 수 있습니다. 혹은 안정자를 갖는 것 자체가 상황을 악화 시키는 것인지도 모릅니다.


안정자는 언제 가치가 있는 것일까요? 아래 코드를 사용하여 몇 가지 직관과 아이디어를 얻을 수 있습니다. 미래에 오류율이 더 낮아진다면 언제든 바뀔 수 있을 것입니다.


In [ ]:
# All the probabilities here only consider errors introduced by the CX gate and assumes they are bit flip errors.

# Probability for a single CX gate
p1 = 0.01
# Probability that there is an error after 2 CX gates (going through stabilizer)
p2 = p1 * (1 - p1) + (1 - p1) * p1
# Probability that the stabilizer shows something wrong even though it is correct
p3 = p1 * p1 + (1 - p1) * (1 - p1) * p1

print("Probability of a single cx having an error: {}".format(p1))
print("Probability of having an error after 2 cx: {:.4f}".format(p2))
print("Probability of the stabilizer showing a non existent error: {:.4f}".format(p3))

## 4 단계: 무엇을 할 수 있을 까요?

여러분이 시도할 수 있는 많은 방법들이 있습니다:


- QASN3와 여러개의 안정자를 사용하는 좋은 논리적인 함수를 찾아내보세요

- 안정자를 재설정하고 다시 사용하는 방법을 생각해 보세요

- 예제 A의 간단한 접근법이 동작하도록 만드는 법을 시도해 보세요(측정이 이루어 지는 타이밍을 바꾸어 보세요) 

- 창의적으로 여러분 만의 방법을 찾아보세요!


위의 아이디어들 모두에서 중요한 것은 그저 맹목적으로 양자컴퓨터에 시도하기 시작하는 것이 아니라, 장치에서 실행해서 얻은 GHZ-상태의 데이터를 먼저 살펴보고 여러분의 아이디어를 검증해보기 위해 사용해 보는 것입니다. 

여러분은 이미 테스트를 위한 자료를 위에서 준비해 두었습니다: GHZ-상태의 출력과 안정자의 출력 결과를 가지고 있습니다. 

만약 여러분의 접근법이 후처리(postprocessing)방식으로 잘 동작하지 않는다면, 모든 연산에 추가 오류율이 존재하는 실제 장비에서 실행한다고 해도, 동작하지 않을 것입니다.

만약, 이미 가지고 있는 데이터로 이론적으로 작동하는 알고리듬이 있고 (테스트에 따라 더 나은 GHZ 상태를 생성한 경우) 아래 코드를 사용하여 오류 정정을 포함한 전체 회로를 실제 양자 장치에서 실행할 수 있습니다.

*힌트: 위의 error_correction 함수를 사용하고 위의 error_corrected_ghz 회로도 생성해야 합니다.*

In [ ]:
# First we transpile the GHZ-state for the actual device
qc_corrected_transpiled = transpile(
    error_corrected_ghz, backend, initial_layout=initial_layout
)

# Now we can run the job
job_corrected = backend.run(
    qc_corrected_transpiled,
    dynamic=True,
    shots=number_of_shots,
    memory=True,
    job_tags=["dynamic", "spring_challenge"],
)

job_id = job_corrected.job_id()
print(job_id)

In [ ]:
job_corrected = provider.retrieve_job(job_id)
job_corrected.status()

In [ ]:
# And get the results back
counts_corrected = job_corrected.result().get_counts()